In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, LeakyReLU, BatchNormalization, Dense, Flatten, UpSampling2D
from tensorflow.keras.models import Model
import numpy as np
import cv2
import os
import json
import glob

# Load Dataset from Train, Valid, and Test folders
def load_dataset(base_path):
    def load_images_and_labels(json_path, image_dir):
      with open(json_path, 'r') as f:
          data = json.load(f)

      images, labels = [], []
      for item in data['images']:
          img_path = os.path.join(image_dir, item['file_name'])
          img = cv2.imread(img_path)
          if img is None:
              continue
          img = cv2.resize(img, (64, 64)) / 127.5 - 1  # Normalize
          images.append(img)

          # Find the corresponding annotation for this image using the image id
          image_id = item['id']
          annotation = next((ann for ann in data['annotations'] if ann['image_id'] == image_id), None)

          if annotation:
              labels.append(annotation['category_id'] - 1)
          else:
              # case where image has no annotation.
              images.pop()

      return np.array(images), np.array(labels)

    train_images, train_labels = load_images_and_labels(os.path.join(base_path, 'train', '_annotations.coco.json'), os.path.join(base_path, 'train')) # Replace with the actual file name
    val_images, val_labels = load_images_and_labels(os.path.join(base_path, 'valid', '_annotations.coco.json'), os.path.join(base_path, 'valid')) # Replace with the actual file name
    test_images, test_labels = load_images_and_labels(os.path.join(base_path, 'test', '_annotations.coco.json'), os.path.join(base_path, 'test')) # Replace with the actual file name

    return train_images, val_images, test_images, train_labels, val_labels, test_labels

# Custom Small Object Detection Model
def build_custom_model():
    inputs = tf.keras.Input(shape=(64, 64, 3))
    x = Conv2D(32, kernel_size=3, activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(64, kernel_size=3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(3, activation='softmax')(x)  # Multi-class classification for Vehicles, Ships, Aircraft

    return Model(inputs, x, name='SmallObjectDetector')

# Training and Evaluation
def train_and_evaluate(model, X_train, y_train, X_val, y_val, X_test, y_test, epochs=10, batch_size=16):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size)

    # Evaluate Model
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    mAP = np.mean(y_pred == y_test)  #mAP calculation
    print(f"Mean Average Precision (mAP): {mAP}")

# Load Data and Train Model
base_path = '/content/SkyFusion'
X_train, X_val, X_test, y_train, y_val, y_test = load_dataset(base_path)
model = build_custom_model()
train_and_evaluate(model, X_train, y_train, X_val, y_val, X_test, y_test)



Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 151s 1s/step - accuracy: 0.8011 - loss: 15.2733 - val_accuracy: 0.4321 - val_loss: 30.6027
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 151s 1s/step - accuracy: 0.9519 - loss: 0.9543 - val_accuracy: 0.5479 - val_loss: 8.2805
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 201s 1s/step - accuracy: 0.9412 - loss: 2.3163 - val_accuracy: 0.7082 - val_loss: 12.7131
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 206s 1s/step - accuracy: 0.9522 - loss: 1.2883 - val_accuracy: 0.9243 - val_loss: 1.3533
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.9383 - loss: 6.7190 - val_accuracy: 0.8820 - val_loss: 3.4173
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 153s 1s/step - accuracy: 0.9825 - loss: 0.2005 - val_accuracy: 0.8886 - val_loss: 1.5648
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 216s 1s/step - accuracy: 0.9778 - loss: 0.2214 - val_accuracy: 0.8196 - val_loss: 18.5208
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.9600 - loss: 1.6281 - val_